# Binary Tempotron
An implementation of the binary tempotron in Julia. 
For further details see [Gütig, R., & Sompolinsky, H. (2006). The tempotron: a neuron that learns spike timing–based decisions. Nature neuroscience, 9(3), 420.](https://www.nature.com/articles/nn1643).

## Imports

In [21]:
using Tempotrons
using Tempotrons.InputGen
using Tempotrons.Plots
using Tempotrons.Optimizers
using Plots
using Plots.PlotMeasures;

## Set parameters

In [22]:
N = 10
T = 500
dt = 1
t = collect(0:dt:T)
λ = 0.01
opt = SGD(λ)
ν = 3
n_samples = 10
n_steps = 5000
tmp = Tempotron(N = N);

## Generate input samples

In [23]:
base_samples = [[PoissonProcess(ν = ν, T = T)
                 for i = 1:N]
                for j = 1:2]
samples = [(x = [SpikeJitter(s, T = T, σ = 5)
                 for s ∈ base_samples[2(j-1)÷n_samples + 1]],
            y = Bool(2(j-1)÷n_samples))
           for j = 1:n_samples];

## Training

In [24]:
# Get the tempotron's output before training
out_b = [tmp(s.x, t = t) for s ∈ samples]

# Train the tempotron
@time for i = 1:n_steps
    s = rand(samples)
    Train!(tmp, s.x, s.y, optimizer = opt)
end

# Get the tempotron's output after training
out_a = [tmp(s.x, t = t) for s ∈ samples];

  0.404307 seconds (4.86 M allocations: 284.981 MiB, 24.32% gc time)


## Plots

In [25]:
plotlyjs(size = (700, 1500))
cols = collect(1:2)#palette(:rainbow, 2)

inp_plots = [PlotInputs(s.x, color = cols[1 + s.y])
             for s ∈ samples]
train_plots = [PlotPotential(tmp, out_b = ob.V, out = oa.V,
                             N_b = length(ob.spikes), N = length(oa.spikes),
                             t = t, color = cols[1 + s.y])
               for (s, ob, oa) ∈ zip(samples, out_b, out_a)]
ip = plot(inp_plots..., layout = (length(inp_plots), 1), link = :all)
tp = plot(train_plots..., layout = (length(train_plots), 1), link = :all)
p = plot(ip, tp, layout = (1, 2), left_margin = 8mm, bottom_margin = 15mm)

<!DOCTYPE html>
 
 
 Plots.jl